In [8]:
%cd ../src/

/mnt/c/Users/Jacob/Desktop/prosjektoppgave/testing/gymming/src


In [9]:
import torch
import torch.nn as nn

### autograd for a

In [10]:
x = torch.ones(10, dtype=torch.float64)
y = torch.zeros(3, dtype=torch.float64) # target tensor
w = torch.randn(10, 3, requires_grad=True, dtype=torch.float64) # weights
b = torch.randn(3, requires_grad=True, dtype=torch.float64) # bias
z = x @ w + b
a = torch.sigmoid(z)
a.retain_grad()
print('a: ', a)
loss = nn.functional.mse_loss(a, y)
print('loss: ', loss.item())
loss.backward()
print('a.grad: ', a.grad)
print('b.grad: ', b.grad)
'''
# gradient descent on a
a.data -= 0.1 * a.grad.data
print('a: ', a)
loss = nn.functional.mse_loss(a, y)
print('loss: ', loss.item())
'''

a:  tensor([0.0437, 0.7890, 0.4500], dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)
loss:  0.2756576226401392
a.grad:  tensor([0.0291, 0.5260, 0.3000], dtype=torch.float64)
b.grad:  tensor([0.0012, 0.0876, 0.0742], dtype=torch.float64)


"\n# gradient descent on a\na.data -= 0.1 * a.grad.data\nprint('a: ', a)\nloss = nn.functional.mse_loss(a, y)\nprint('loss: ', loss.item())\n"

### finite differences for b

In [11]:
# estimate b gradients with finite differences
with torch.no_grad():
    h = 1e-10
    b1 = b.clone()
    for i in range(3):
        b1[i] += h
        z1 = x @ w + b1
        a1 = torch.sigmoid(z1)
        loss1 = nn.functional.mse_loss(a1, y)
        print('finite difference: ', (loss1.item() - loss.item()) / h)
        b1[i] -= h

finite difference:  0.0012156942119645464
finite difference:  0.08756217972916147
finite difference:  0.0742483852178566


### finite differences for a

In [12]:
# estimate a gradients with finite differences
with torch.no_grad():
    h = 1e-10
    a1 = a.clone()
    for i in range(3):
        a1[i] += h
        loss1 = nn.functional.mse_loss(a1, y)
        print('finite difference: ', (loss1.item() - loss.item()) / h)
        a1[i] -= h

finite difference:  0.029118929489868606
finite difference:  0.526019783286813
finite difference:  0.29999724926454974


### autograd for cav

In [13]:
cav = torch.tensor([0.0, 0.5, 1.0], dtype=torch.float64)

x = torch.ones(10, dtype=torch.float64)
y = torch.zeros(3, dtype=torch.float64) # target tensor
w = torch.randn(10, 3, requires_grad=True, dtype=torch.float64) # weights
b = torch.randn(3, requires_grad=True, dtype=torch.float64) # bias
z = x @ w + b
a = torch.sigmoid(z)
a.retain_grad()
print('a: ', a)
loss = nn.functional.mse_loss(a, y)
print('loss: ', loss.item())
loss.backward()
print('a.shape: ', a.shape)
print('cav.shape: ', cav.shape)
print('a.grad: ', a.grad)
cav_grad = a.grad @ cav
print('cav_grad: ', cav_grad)
# update a with cav_grad
with torch.no_grad():
    a1 = a.clone()
    a1 -= 0.5 * cav * cav_grad
    print('a1: ', a1)
    print('a diff: ', a1 - a)
    loss1 = nn.functional.mse_loss(a1, y)
    print('loss1: ', loss1.item())

a:  tensor([0.9992, 0.1203, 0.3763], dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)
loss:  0.38480777941211314
a.shape:  torch.Size([3])
cav.shape:  torch.Size([3])
a.grad:  tensor([0.6661, 0.0802, 0.2509], dtype=torch.float64)
cav_grad:  tensor(0.2910, dtype=torch.float64)
a1:  tensor([0.9992, 0.0476, 0.2308], dtype=torch.float64)
a diff:  tensor([ 0.0000, -0.0728, -0.1455], dtype=torch.float64)
loss1:  0.3512878126060858


### finite differences for cav

In [14]:
# d L/d cav, when cav is a direction in a?
# estimate cav gradients with finite differences
with torch.no_grad():
    h = 1e-10
    a1 = a.clone()
    perturb = h * cav
    print('perturb: ', perturb)
    a1 += perturb
    loss1 = nn.functional.mse_loss(a1, y)
    print('loss0: ', loss.item())
    print('loss1: ', loss1.item())
    print('finite difference cav: ', (loss1.item() - loss.item()) / h)

perturb:  tensor([0.0000e+00, 5.0000e-11, 1.0000e-10], dtype=torch.float64)
loss0:  0.38480777941211314
loss1:  0.38480777944121325
finite difference cav:  0.2910011120960121
